In [1]:
import json
import re

In [2]:
base_path = "module1/data/json/path"
gpt4_path = "gpt4/data/json/path"
llama_path = "llama2_7B/data/json/path"
opt6_path = "opt6_3b/data/json/path"
opt1_path = "opt1_3b/data/json/path"
frozen_path = "frozen/data/json/path"

with open(base_path, "r") as f:
    base = json.load(f)
    if 'data' in base:
        base = base['data']

with open(gpt4_path, "r") as f:
    gpt4 = json.load(f)
    if 'data' in gpt4:
        gpt4 = gpt4['data']
    
with open(llama_path, "r") as f:
    llama = json.load(f)
    if 'data' in llama:
        llama = llama['data']

with open(opt6_path, "r") as f:
    opt = json.load(f)
    if 'data' in opt:
        opt = opt['data']

with open(opt1_path, "r") as f:
    opt1 = json.load(f)
    if 'data' in opt1:
        opt1 = opt1['data']

with open(frozen_path, "r") as f:
    frozen = json.load(f)
    if 'data' in frozen:
        frozen = frozen['data']

In [3]:
answer_dict_for_gpt4 = {
    'Normal Cognition': 'Answer: (C) Normal Cognition',
    'Mild Cognitive Impairment': 'Answer: (B) Mild Cognitive Impairment',
    'Alzhimer\'s Disease': 'Answer: (A) Alzheimer\'s disease'
}
answer_dict = {
    'Normal Cognition': 'C',
    'Mild Cognitive Impairment': 'B',
    'Alzhimer\'s Disease': 'A'
}
def get_answer(answer):
    ans = re.findall(r'\([A-D]\)', answer)
    ans = [a[1:-1] for a in ans]
    if len(ans) == 0:
        return None
    return ans[0]

In [4]:
save_data = []
all_corect_idx_list = []
all_wrong_idx_list = []
for d in base:
    if d['id'] == 452 or d['id'] == 707:
        continue
    temp = {
        'id': d['id'],
        'input': d['input'],
        'patient_data': "\n".join(d['patient_data'].split(' - ')),
        'org_label': d['org_label'],
        'gpt4_gold': d['label'],
    }
    org_answer = answer_dict[d['org_label']]
    for g in gpt4:
        if g['id'] == d['id']:
            tmp = g['prediction'].split("Diagnosis: ")[-1].strip()
            if "." in tmp:
                tmp = tmp.split(".")[0]
            tmp_pred = answer_dict_for_gpt4[tmp]
            tmp2 = g['prediction'].split("Diagnosis: ")[:-1][0] + "\n" + tmp_pred
            temp['gpt4_CoT'] = tmp2
            gpt4_answer = get_answer(temp['gpt4_CoT'])
            break
        
    for l in llama:
        if l['id'] == d['id']:
            temp['llama2'] = l['prediction']
            llama_answer = get_answer(temp['llama2'])
            break
        
    for o in opt:
        if o['id'] == d['id']:
            temp['opt6'] = o['prediction']
            opt_answer = get_answer(temp['opt6'])
            break
    
    for o1 in opt1:
        if o1['id'] == d['id']:
            temp['opt1'] = o1['prediction']
            opt1_answer = get_answer(temp['opt1'])
            break
    
    for f in frozen:
        if f['id'] == d['id']:
            temp['resnet50_llama2'] = f['prediction']
            resnet50_llama2_answer = get_answer(temp['resnet50_llama2'])
            break
    save_data.append(temp)
    if org_answer == gpt4_answer and org_answer == llama_answer and org_answer == opt_answer and org_answer == opt1_answer and org_answer == resnet50_llama2_answer:
        all_corect_idx_list.append(d['id'])
    if org_answer != gpt4_answer and org_answer != llama_answer and org_answer != opt_answer and org_answer != opt1_answer and org_answer != resnet50_llama2_answer:
        all_wrong_idx_list.append(d['id'])

print(len(save_data))
    

757


In [5]:
save_path = "save_total/eval/json/path"

with open(save_path, "w") as f:
    json.dump(save_data, f, indent=4, ensure_ascii=False)

In [6]:
print(len(all_corect_idx_list))
print(len(all_wrong_idx_list))

295
57


In [7]:
save_correct_path = "all/correct/json/data/path"
save_wrong_path = "all/wrong/json/data/path"

In [8]:
correct_data = []
wrong_data = []
for d in save_data:
    if d['id'] in all_corect_idx_list:
        correct_data.append(d)
    if d['id'] in all_wrong_idx_list:
        wrong_data.append(d)

print(f"Correct data: {len(correct_data)}")
print(f"Wrong data: {len(wrong_data)}")

Correct data: 295
Wrong data: 57


In [9]:
# save data
with open(save_correct_path, 'w') as f:
    json.dump(correct_data, f, indent=4, ensure_ascii=False)

with open(save_wrong_path, 'w') as f:
    json.dump(wrong_data, f, indent=4, ensure_ascii=False)


In [10]:
# statistics
correct_dict = {
    'Alzhimer\'s Disease': 0,
    'Mild Cognitive Impairment': 0,
    'Normal Cognition': 0
}
correct_idx_dict = {
    'Alzhimer\'s Disease': [],
    'Mild Cognitive Impairment': [],
    'Normal Cognition': []
}
wrong_dict = {
    'Alzhimer\'s Disease': 0,
    'Mild Cognitive Impairment': 0,
    'Normal Cognition': 0
}
wrong_idx_dict = {
    'Alzhimer\'s Disease': [],
    'Mild Cognitive Impairment': [],
    'Normal Cognition': []
}

In [11]:
for idx, d in enumerate(correct_data):
    correct_dict[d['org_label']] += 1
    correct_idx_dict[d['org_label']].append(idx)
    

for idx, d in enumerate(wrong_data):
    wrong_dict[d['org_label']] += 1
    wrong_idx_dict[d['org_label']].append(idx)
    
print(correct_dict)
print(wrong_dict)

{'Dementia': 152, 'Mild Cognitive Impairment': 31, 'Normal Cognition': 112}
{'Dementia': 14, 'Mild Cognitive Impairment': 37, 'Normal Cognition': 6}


In [33]:
correct_idx_list = correct_idx_dict["Alzhimer\'s Disease"][4:8]
wrong_idx_list = wrong_idx_dict["Alzhimer\'s Disease"][4:8]

for val in correct_idx_dict["Mild Cognitive Impairment"][3:6] : 
    correct_idx_list.append(val) 
for val in correct_idx_dict["Normal Cognition"][3:6]:
    correct_idx_list.append(val)


for val in wrong_idx_dict["Mild Cognitive Impairment"][3:6] : 
    wrong_idx_list.append(val)
for val in wrong_idx_dict["Normal Cognition"][3:6]:
    wrong_idx_list.append(val)


In [34]:
print(len(correct_idx_list))
print(len(wrong_idx_list))

10
10


In [35]:
# using correct_data and correct_idx_list
correct_new_data = [correct_data[i] for i in correct_idx_list]
wrong_new_data = [wrong_data[i] for i in wrong_idx_list]


In [37]:
print(len(correct_new_data))
print(len(wrong_new_data))

10
10


In [38]:
total_data = [i for i in correct_new_data]
total_data.extend([i for i in wrong_new_data])
print(len(total_data))
# shuffle total_data
import random
random.shuffle(total_data)

20


In [39]:
new_data = []
for idx, d in enumerate(total_data):
    temp = {
        "id": idx+1,
        "org_id": d["id"],
        "patient_data": d["patient_data"],
        "org_label": "Alzheimer's Disease" if d["org_label"] == 'Dementia' else d["org_label"],
        "GPT4_Gold": d["gpt4_gold"],
        "GPT4_CoT": d["gpt4_CoT"],
        "LLaMA2_7B": d["llama2"],
        "OPT6B": d["opt6"],
        "OPT1B": d["opt1"],
        "resnet50_llama2": d["resnet50_llama2"],
    }
    new_data.append(temp)

print(len(new_data))

20


In [40]:
# save data
save_path = "Final/data/json/path"
with open(save_path, "w") as f:
    json.dump(new_data, f, indent=4, ensure_ascii=False)